<a href="https://colab.research.google.com/github/debjde6400/cerec-2/blob/main/CoreNLP_Server_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**CoreNLP Server tunnelling**

Through this notebook we will establish a CoreNLP server for Cerec 2 system. It uses Ngrok tunnelling to emulate the server through localhost:9002.

# Mount google drive

Use 'Mount Drive' option from the sidebar to connect your google drive.

##Importing required packages

In [1]:
import os, sys
import subprocess
import json
import time
import requests

## Get and set the core nlp server

(Here CoreNLP 4.2.2 has been used.)

In [2]:
! wget http://nlp.stanford.edu/software/stanford-corenlp-latest.zip
! unzip stanford-corenlp-latest.zip

--2021-07-10 19:34:44--  http://nlp.stanford.edu/software/stanford-corenlp-latest.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/software/stanford-corenlp-latest.zip [following]
--2021-07-10 19:34:44--  https://nlp.stanford.edu/software/stanford-corenlp-latest.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://downloads.cs.stanford.edu/nlp/software/stanford-corenlp-latest.zip [following]
--2021-07-10 19:34:45--  https://downloads.cs.stanford.edu/nlp/software/stanford-corenlp-latest.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response..

In [3]:
% cd stanford-corenlp-4.2.2

/content/stanford-corenlp-4.2.2


## Setting up server

Here we set up the server and generate the url to it. There are 2 ways to generate such a server, which are discussed below.

### Way 1

These are the helper methods used here.

In [4]:
def _get_ngrok_tunnel():
    while True:
        try:
            tunnels_json = requests.get("http://localhost:4040/api/tunnels").content
            public_url = json.loads(tunnels_json)['tunnels'][0]['public_url']
            return public_url
        except Exception:
            print("Can't get public url, retrying...")
            time.sleep(2)


def _warmup_ngrok_tunnel(public_url):
    while requests.get(public_url).status_code >= 500:
        print("Tunnel is not ready, retrying...")
        time.sleep(2)

Set up server using net-tools and ngrok.

In [5]:
def expose_port_on_colab(port):
    '''os.system("apt-get install net-tools")
    # check that port is open
    while not (":{} ".format(port) in str(subprocess.check_output("netstat -vatn", shell=True))):
        print("Port {} is closed, retrying...".format(port))
        time.sleep(2)'''

    # run ngrok
    get_ipython().system_raw("wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip") #(only for 1st time)
    get_ipython().system_raw("unzip ngrok-stable-linux-amd64.zip")
    get_ipython().system_raw("./ngrok authtoken 1tRpHQnGTioq0wCWmLIWpXLNRmt_7g77xtvwmBAUmuTG52gs7")
    LOGDIR = '/tmp/log'
    get_ipython().system_raw('java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9002 -timeout 300000 &'.format(LOGDIR))
    get_ipython().system_raw("./ngrok http {0} &".format(port))
    public_url = _get_ngrok_tunnel()
    _warmup_ngrok_tunnel(public_url)

    print("Open {0} to access your {1} port".format(public_url, port))

Expose port by calling the above method.

In [6]:
expose_port_on_colab(9002)

Can't get public url, retrying...
Open http://2f21e1729c5f.ngrok.io to access your 9002 port


## Close server

In [7]:
get_ipython().system_raw('./ngrok kill')